In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
sess = tf.InteractiveSession()

In [6]:
# 卷积函数
# strides: NHWC 长度为 4 的列表： [batch, height, width, channel]
def conv2d(x, W, stride=1, padding='SAME'):
    strides = np.r_[1, stride, stride, 1].tolist()
    return tf.nn.conv2d(x, W, strides=strides, padding=padding)

# 最大池化函数：2*2像素 -> 1*1,保留灰度最大的像素
# strides
def max_pool(x, size=2, stride=2, padding='SAME'):
    ksize = np.r_[1, size, size, 1].tolist()
    strides = np.r_[1, stride, stride, 1].tolist()
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding=padding)

In [7]:
X = tf.placeholder(dtype=tf.float32, shape=[784, None])
y_ = tf.placeholder(dtype=tf.float32, shape=[10, None])
# 变回具有空间结构信息的 tensor， 其中 -1表示样本数量不确定
x_image = tf.reshape(tensor=X, shape=[-1, 28, 28, 1])

In [10]:
# layer1(conv1)  input: 28 * 28 * 1 -> output: 14 * 14 * 32
conv_shape = [5, 5, 1, 32]
conv_stride = 1
pool_size = 2
pool_stride = 2
## conv1 input:28 * 28 * 1 -> output: 28 * 28 * 32
W_conv1 = weight_variables(conv_shape)  # output:32
b_conv1 = bias_variable([conv_shape[-1]])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, stride=conv_stride, padding='SAME') + b_conv1)
## pool1 input: 28 * 28 * 32 -> output: 14 * 14 * 32
h_pool1 = max_pool(h_conv1, size=pool_size, stride=pool_stride, padding='SAME')

# layer2(conv2) input: 14 * 14 * 32 -> output: 7 * 7 * 64
conv_shape = [5, 5, 32, 64]
conv_stride = 1
pool_size = 2
pool_stride = 2
## conv2 input: 14 * 14 * 32 -> output: 14 * 14 * 32
W_conv2 = weight_variables(conv_shape)
b_conv2 = bias_variable([conv_shape[-1]])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, stride=conv_stride, padding='SAME') + b_conv2)
## pool2 input: 14 * 14 * 64 -> 7 * 7 * 64
h_pool2 = max_pool(h_conv2, size=pool_size, stride=pool_stride, padding='SAME')

# layer3(FC1) input: 7 * 7 * 64 -> output: 1024 * m
h_pool2_flat = tf.reshape(tensor=h_pool2, shape=[7 * 7 * 64, -1])
W_fc1 = weight_variables(shape=[1024, 7 * 7 * 64])
b_fc1 = bias_variable(shape=[1024, 1])
h_fc1 = tf.nn.relu(W_fc1 @ h_pool2_flat + b_fc1)

# DropOut: input: 1024 * m -> output: 1024 * m
keep_prob = tf.placeholder(dtype=tf.float32)
h_fc1_drop = tf.nn.dropout(x=h_fc1, keep_prob=keep_prob)

# softmax: input: 1024 * m -> output: 10 * m
W_fc2 = weight_variables(shape=[10, 1024])
b_fc2 = bias_variable(shape=[10, 1])
y_conv = tf.nn.softmax(W_fc2 @ h_fc1_drop + b_fc2)

# 损失函数计算
cost = tf.reduce_mean(-tf.reduce_sum(tf.multiply(y_, tf.log(y_conv)), axis=0))
# 训练的优化器设定
train_step = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cost)
# 其他预测指标的设定
correct_prediction = tf.equal(tf.argmax(y_conv, axis=0), tf.argmax(y_, axis=0))
acc = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

In [13]:
file_writer = tf.summary.FileWriter(logdir="F:/tmp/", graph=sess.graph)

In [ ]:
tf.global_variables_initializer().run()
test_holder = {X: mnist.test.images.T, y_: mnist.test.labels.T, keep_prob: 1.0}
display_step = 100
test_acc_list = []
for i in range(20000):
    batch_xs, batch_ys = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={X: batch_xs.T, y_: batch_ys.T, keep_prob: 0.75})
    if i % display_step == 0:
        test_acc = acc.eval(feed_dict=test_holder)
        test_acc_list.append(test_acc)
        print("epoch %d, test_acc: %.4f" % (i, test_acc))